In [79]:
#from dataloaders import Dataloader as DL
import os
from pathlib import Path
import h5py
import pandas as pd
import numpy as np
import arpespythontools as arp
import matplotlib.pyplot as plt
from ipywidgets import widgets, Layout, interactive_output
import astropy 

In [80]:
#FLAWS: IF THEY CHANGE ORGANIZATION OF HDF5 DATA, IT WILL NOT WORK!
#HF FIlE AND MAP_DATA!!!
global hf
global map_data2
global phi
global energy
global theta

hf = h5py.File("20230317_00029.h5", 'r')

#FINDING MAP_DATANAME
keys = list(hf.keys())
map_data_name = str(list(hf['2D_Data'].keys())[0])
#READING IN MAP_DATA
map_data2 = hf["2D_Data"][map_data_name]
map_data2 = np.array(map_data2[:][:][:])

#GETTING THE SCALEDELTA AND OFFSET RIGHT
if hf["2D_Data"][map_data_name].attrs["unitNames"][0] == 'eV':
    offset_energy = hf["2D_Data"][map_data_name].attrs["scaleOffset"][0]
    scaleDelta_energy = hf["2D_Data"][map_data_name].attrs["scaleDelta"][0]
    offset_theta = hf["2D_Data"][map_data_name].attrs["scaleOffset"][1]
    scaleDelta_theta = hf["2D_Data"][map_data_name].attrs["scaleDelta"][1]
elif hf["2D_Data"][map_data_name].attrs["unitNames"][1] == 'eV':
    offset_energy = hf["2D_Data"][map_data_name].attrs["scaleOffset"][1]
    scaleDelta_energy = hf["2D_Data"][map_data_name].attrs["scaleDelta"][1]
    offset_theta = hf["2D_Data"][map_data_name].attrs["scaleOffset"][0]
    scaleDelta_theta = hf["2D_Data"][map_data_name].attrs["scaleDelta"][0]

#CREATES AXIS FOR PHI, ENERGY, AND THETA
phi = np.array(hf["0D_Data"]["Slit Defl"][:])
energy = np.array([])
theta = np.array([])

#ASSUMING (E, THETA, PHI); MUST BE CHANGED IF NOT
for i in range(map_data2.shape[0]):
    energy = np.append(energy, [offset_energy + scaleDelta_energy*i])
        
for i in range(map_data2.shape[1]):
    theta = np.append(theta, offset_theta + scaleDelta_theta*i)

#CENTER DATA
theta_v = 0.181*(theta - 533)

# ANGLE TO WAVEVECTOR

In [81]:
def angle_to_wavevector_binding(E_pho, work_f, theta, phi):
    #CONVERTS PIXELS TO THETA
    global theta_v
    
    #print(theta_v)
    E_k = E_pho - work_f
    k = 0.512*np.sqrt(E_k)
    theta_rad = np.radians(theta_v)
    phi_rad = np.radians(phi)
    
    #K_x and K_Y 
    k_x = k*np.sin(theta_rad)
    k_y = k*np.sin(phi_rad)
    
    #k_x = np.degrees(k_x)
    #k_y = np.degrees(k_y)
    return k_x, k_y
        
    
global k_x
global k_y

k_x, k_y = angle_to_wavevector_binding(85, 4, theta_v, phi)

# K_Y Vs. K_X

In [82]:
def plt_plot(Vmax = 16226, Emin = -0.44, Emax = -0.41):
    global emin1
    global emax2
    global vmax1
    
    emin1 = Emin
    emax2 = Emax
    vmax1 = Vmax
    
    surface1 = arp.plane_slice(map_data2, energy, Emin, Emax)
    #diff2 = arp.laplacian(surface1, phi, theta)
    plt.figure(figsize = (8, 6))
    fig = plt.imshow(surface1, aspect = "auto", vmax = Vmax, origin = 'lower', 
                     extent = [k_x[0], k_x[-1], k_y[0], k_y[-1]], cmap = 'gray')
    
    plt.colorbar()
    plt.xlabel("Wavevector (k_x)")
    plt.ylabel("Wavevector (k_y)")
    

widgets.interact(plt_plot, Vmax = (0, 100000, 0.01), Emin = (energy[0], energy[-1], 0.01), 
                 Emax = (energy[0], energy[-1], 0.01) )

interactive(children=(FloatSlider(value=16226.0, description='Vmax', max=100000.0, step=0.01), FloatSlider(val…

<function __main__.plt_plot(Vmax=16226, Emin=-0.44, Emax=-0.41)>

In [44]:
def plt_plot(Vmax = vmax1, x1 = k_x[0], x2 = k_x[-1], y1 = k_y[0], y2 = k_y[-1]):
    surface1 = arp.plane_slice(map_data2, energy, emin1, emax2)
    #diff2 = arp.laplacian(surface1, phi, theta)
    plt.figure(figsize = (8, 6))
    fig = plt.imshow(surface1, aspect = "auto", vmax = Vmax, origin = 'lower', 
                     extent = [k_x[0], k_x[-1], k_y[0], k_y[-1]], cmap = 'gray')
    
    plt.colorbar()
    plt.xlabel("Wavevector (k_x)")
    plt.ylabel("Wavevector (k_y)")
    
    plt.xlim(x1, x2)
    plt.ylim(y1, y2)

widgets.interact(plt_plot, Vmax = (0, 100000, 0.01),
                x1 = (k_x[0], k_x[-1], 0.01),
                x2 = (k_x[0], k_x[-1], 0.01),
                y1 = (k_y[0], k_y[-1], 0.01),
                y2 = (k_y[0], k_y[-1], 0.01))

interactive(children=(FloatSlider(value=44446.47000000001, description='Vmax', max=100000.0, step=0.01), Float…

<function __main__.plt_plot(Vmax=44446.47000000001, x1=-4.504441695914476, x2=4.491807287242438, y1=-1.1926381598324156, y2=1.1926381598324156)>

# Energy Vs. K_Y

In [35]:
def plt_plot(Vmax = 50000, k_x_min = -1, k_x_max = 360):
    global k_x1
    global k_x2
    global vmax2
    
    k_x1 = k_x_min
    k_x2 = k_x_max
    vmax2 = Vmax
    
    surface1 = arp.plane_slice(map_data2.transpose([1, 0,2]), k_x, k_x_min, k_x_max)
    plt.figure(figsize = (8, 6))
    plt.imshow(surface1, aspect = "auto", vmax = Vmax, origin = 'lower', extent = [k_y[0], k_y[-1], energy[0], energy[-1]], cmap = 'gray')
    plt.xlabel("Wavevector (k_y)")
    plt.ylabel("Energy")
    plt.colorbar()
    

widgets.interact(plt_plot, Vmin = 0, Vmax = (0, 1000000, 0.01), k_x_min = (k_x[0], k_x[-1], 0.01), 
                 k_x_max = (k_x[0], k_x[-1], 0.01))

interactive(children=(FloatSlider(value=50000.0, description='Vmax', max=1000000.0, step=0.01), FloatSlider(va…

<function __main__.plt_plot(Vmax=50000, k_x_min=-1, k_x_max=360)>

In [37]:
def plt_plot(Vmax = vmax2, x1 = k_y[0], x2 = k_y[-1], y1 = energy[0], y2 = energy[-1]):
    surface1 = arp.plane_slice(map_data2.transpose([1, 0,2]), k_x, k_x1, k_x2)
    plt.figure(figsize = (8, 6))
    plt.imshow(surface1, aspect = "auto", vmax = Vmax, origin = 'lower', extent = [k_y[0], k_y[-1], energy[0], energy[-1]], cmap = 'gray')
    plt.xlabel("Wavevector (k_y)")
    plt.ylabel("Energy")
    plt.colorbar()
    
    plt.xlim(x1, x2)
    plt.ylim(y1, y2)
    

widgets.interact(plt_plot, Vmin = 0, Vmax = (0, 1000000, 0.01),
                x1 = (k_y[0], k_y[-1], 0.01),
                x2 = (k_y[0], k_y[-1], 0.01),
                y1 = (energy[0], energy[-1], 0.01),
                y2 = ((energy[0], energy[-1], 0.01)))

interactive(children=(FloatSlider(value=622472.44, description='Vmax', max=1000000.0, step=0.01), FloatSlider(…

<function __main__.plt_plot(Vmax=622472.44, x1=-1.1926381598324156, x2=1.1926381598324156, y1=-2.066022544283414, y2=-0.10950080515297889)>

# Energy Vs K_X

In [30]:
def plt_plot(Vmax = 35480, Pmin = -1, Pmax = 2):
    global pmin
    global pmax
    global vmax
    
    pmin3 = Pmin
    pmax3 = Pmax
    vmax3 = Vmax
    
    surface1 = arp.plane_slice(map_data2.transpose([2, 0, 1]), phi, Pmin, Pmax)
    plt.figure(figsize = (8, 6))
    plt.imshow(surface1, aspect = "auto", vmax = Vmax, origin = 'lower', extent = [k_x[0], k_x[-1], energy[0], energy[-1]], cmap = 'gray')
    plt.xlabel("Wavevector (k_x)")
    plt.ylabel("Energy")

    #plt.xlim(-2, 2)
    #plt.ylim(-1.3, -0.25)
    plt.colorbar()
    
    

widgets.interact(plt_plot, Vmin = 0, Vmax = (0, 100000, 0.01), Pmin = (phi[0], phi[-1], 0.01), 
                 Pmax = (phi[0], phi[-1], 0.01))

interactive(children=(FloatSlider(value=35480.0, description='Vmax', max=100000.0, step=0.01), FloatSlider(val…

<function __main__.plt_plot(Vmax=35480, Pmin=-1, Pmax=2)>

In [31]:
def plt_plot(Vmax = vmax3, x1 = k_x[0], x2 = k_x[-1], y1 = energy[0], y2 = energy[-1]):
    surface1 = arp.plane_slice(map_data2.transpose([2, 0, 1]), phi, pmin3, pmax3)
    plt.figure(figsize = (8, 6))
    plt.imshow(surface1, aspect = "auto", vmax = Vmax, origin = 'lower', extent = [k_x[0], k_x[-1], energy[0], energy[-1]], cmap = 'gray')
    plt.xlabel("Wavevector (k_x)")
    plt.ylabel("Energy")
    
    plt.xlim(x1, x2)
    plt.ylim(y1, y2)
    
    plt.colorbar()
    
    

widgets.interact(plt_plot, Vmin = 0, Vmax = (0, 100000, 0.01), 
                 x1 = (k_x[0], k_x[-1], 0.01), 
                 x2 = (k_x[0], k_x[-1], 0.01), 
                 y1 = (energy[0], energy[-1], 0.01),
                 y2 = (energy[0], energy[-1], 0.01))

interactive(children=(FloatSlider(value=66925.67000000001, description='Vmax', max=100000.0, step=0.01), Float…

<function __main__.plt_plot(Vmax=66925.67000000001, x1=-4.504441695914476, x2=4.491807287242438, y1=-2.066022544283414, y2=-0.10950080515297889)>

# 1D Images

# Intesity Vs. K_Y

In [83]:
def plot1d(k_x1 = 1, k_x2 = 2):
    surface = arp.plane_slice(map_data2, energy, emin1, emax2)
    ky_line = arp.line_profile(surface.T, k_x, k_x1, k_x2)
    print(ky_line.shape)
    plt.plot(k_y, ky_line/max(ky_line))
    plt.ylabel("INTENSITY")
    plt.xlabel("Wavevector(k_y)")
    plt.show()

widgets.interact(plot1d, k_x1 = (k_x[0], k_x[-1], 0.01), k_x2= (k_x[0], k_x[-1], 0.01))

interactive(children=(FloatSlider(value=1.0, description='k_x1', max=4.491807287242438, min=-4.504441695914476…

<function __main__.plot1d(k_x1=1, k_x2=2)>

# INTENSITY VS. K_X

In [69]:
def plot1d(k_y1 = 1, k_y2 = 2):
    surface = arp.plane_slice(map_data2, energy, emin1, emax2)
    kx_line = arp.line_profile(surface, k_y, k_y1, k_y2)
    print(kx_line.shape)
    plt.plot(k_x, kx_line/max(kx_line))
    plt.ylabel("INTENSITY")
    plt.xlabel("Wavevector(k_x)")
    plt.show()

widgets.interact(plot1d, k_y1 = (k_y[0], k_y[-1], 0.01), k_y2= (k_y[0], k_y[-1], 0.01))

interactive(children=(FloatSlider(value=1.0, description='k_y1', max=1.1926381598324156, min=-1.19263815983241…

<function __main__.plot1d(k_y1=1, k_y2=2)>

# Intensity Vs. E

In [77]:
def plot1d(k_y1 = 1.08, k_y2 = 1.19):
    surface = arp.plane_slice(map_data2.transpose([1, 0, 2]), k_x, k_x1, k_x2)
    energy_line = arp.line_profile(surface, k_y, k_y1, k_y2)
    print(energy_line.shape)
    plt.plot(energy, energy_line/max(energy_line))
    plt.ylabel("INTENSITY")
    plt.xlabel("Energy (eV))")
    plt.show()

widgets.interact(plot1d, k_y1 = (k_y[0], k_y[-1], 0.01), k_y2= (k_y[0], k_y[-1], 0.01))

interactive(children=(FloatSlider(value=1.08, description='k_y1', max=1.1926381598324156, min=-1.1926381598324…

<function __main__.plot1d(k_y1=1.08, k_y2=1.19)>

# WATERFALL PLOTS